In [19]:
# 操作 browser 的 驅動程式
from selenium import webdriver

# 期待元素出現要透過什麼方式指定，經常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制停止/強制等待 (程式執行期間休息一下)
from time import sleep

# 建立資料夾與執行檔案相關操作
import os

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                
my_options.add_argument("--start-maximized")         
my_options.add_argument("--incognito")               
my_options.add_argument("--disable-popup-blocking")  
my_options.add_argument("--disable-notifications")   

# os.getcwd() # 拿到絕對路徑

#透過絕對路徑新增資料夾
folderPath = os.path.join(os.getcwd(), 'files')
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 自訂下載路徑 (不會詢問下載位置)
my_options.add_experimental_option("prefs", {
    "download.default_directory": folderPath,
    "profile.default_content_settings.popups": 0,
    "download.prompt_for_download": False,
    # "download.directory_upgrade": True,
    # "safebrowsing_for_trusted_sources_enabled": False,
    # "safebrowsing.enabled": False,
    # "plugins.always_open_pdf_externally": True
})

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options
)

# 走訪網址
url = 'https://www.twse.com.tw/zh/page/trading/fund/TWT38U.html'

In [20]:
# 走訪頁面
def visit():
    driver.get(url)

# 選取下拉式選單的項目
def setDropDownMenu(year, value, index):
    sleep(1)

    # 選擇 select[name="yy"] 元素，並依 option 的 innerText 來進行選取
    yy = Select(driver.find_element(By.CSS_SELECTOR, 'select[name=yy]'))
    yy.select_by_visible_text(f'民國 {year} 年')

    # 選擇 select[name="mm"] 元素
    mm = Select(driver.find_element(By.CSS_SELECTOR, 'select[name=mm]'))
    mm.select_by_value(str(value))

    # 選擇 select[name="dd"] 元素
    dd = Select(driver.find_element(By.CSS_SELECTOR, 'select[name=dd]'))
    dd.select_by_index(index)

    # 按下查詢
    driver.find_element(
        By.CSS_SELECTOR,'div.submit'
    ).click()

# 下載檔案
def download(year, value, index):
    # 下載 csv
    year = 1911 + year
    value = '0' + str(value) if value < 10 else str(value)
    index = '0' + str(index + 1) if (index + 1) < 10 else str(index + 1)
    date = f'{year}{value}{index}'
    os.system(
        f'curl "https://www.twse.com.tw/rwd/zh/fund/TWT38U?date={date}&response=csv" -o {folderPath}/{date}.csv'
    )

    # 擷圖
    driver.save_screenshot(
        f"{folderPath}/{date}.png"
    )

# 關閉瀏覽器
def close():
    driver.quit()


In [21]:
visit()

# 指定年、月、日，檢視查詢結果，並下載 csv
year = 112
value = 10
index = 12
setDropDownMenu(year, value, index)
download(year, value, index)

In [22]:
close()